In [2]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-test-dan-train/train.csv
/kaggle/input/data-test-dan-train/test.csv
/kaggle/input/ag-news-classification-dataset/train.csv
/kaggle/input/ag-news-classification-dataset/test.csv


In [33]:
import pandas as pd

# Panggil Dataset
file_path = '/kaggle/input/data-test-dan-train/train.csv'
df = pd.read_csv(file_path)

# Rename Kolom
df.columns = ['class index', 'Title', 'Description']

# Save
output_path = 'train_updated.csv'
df.to_csv(output_path, index=False)
print(f"File berhasil diubah dan disimpan sebagai {output_path}")

File berhasil diubah dan disimpan sebagai train_updated.csv


In [34]:
import pandas as pd

# Import Data
file_path = '/kaggle/input/data-test-dan-train/test.csv'
df = pd.read_csv(file_path)

# Rename kolom
df.columns = ['class index', 'Title', 'Description']

# Save 
output_path = 'test_updated.csv'
df.to_csv(output_path, index=False)

print(f"File berhasil diubah dan disimpan sebagai {output_path}")


File berhasil diubah dan disimpan sebagai test_updated.csv


In [35]:
train = pd.read_csv('/kaggle/working/train_updated.csv')
test = pd.read_csv('/kaggle/working/test_updated.csv')

In [36]:
train.head()

,class index,Title,Description
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...


In [39]:
train['class index'].value_counts()

class index
4    30000
2    30000
1    30000
3    29999
Name: count, dtype: int64

In [40]:
train.shape

(119999, 3)

In [41]:
test.head()

,class index,Title,Description
0,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
1,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
2,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
3,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
4,4,Open Letter Against British Copyright Indoctri...,The British Department for Education and Skill...


In [42]:
def count_words(text):
    return len(text.split())
train['word_count'] = train['Description'].apply(count_words)

In [43]:
train['word_count'].mean()

31.062850523754364

In [45]:
def get_data(df):
    x = df['Description']
    y = df['class index']
    return x,y

In [46]:
X_train,y_train = get_data(train)
X_test,y_test = get_data(test)

In [47]:
y_train = y_train - 1
y_test = y_test - 1

In [48]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [49]:
y_test

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [50]:
from transformers import AutoTokenizer,TFBertModel

In [51]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are 

In [52]:
max_len = 32
X_train = tokenizer(
    text = X_train.tolist(),
    max_length = max_len,
    return_tensors='tf',
    add_special_tokens = True,
    truncation = True,
    padding = True,
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)
X_test = tokenizer(
    text = X_test.tolist(),
    max_length = max_len,
    return_tensors='tf',
    add_special_tokens = True,
    truncation = True,
    padding = True,
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [53]:
input_ids = X_train['input_ids']
attention_mask = X_train['attention_mask']

## Model Building

In [54]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense     

In [55]:
import tensorflow as tf
from tensorflow.keras import Model, Input, Layer
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Dropout, GlobalMaxPooling1D

class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert_model = TFBertModel.from_pretrained("bert-base-uncased")

    def call(self, inputs):
        input_ids, attention_mask = inputs
        return self.bert_model(input_ids, attention_mask=attention_mask)[0]

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

embeddings = BertLayer()([input_ids, input_mask])

out = GlobalMaxPooling1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)
y = Dense(4, activation='softmax')(out)

model = Model(inputs=[input_ids, input_mask], outputs=y)

model.layers[2].trainable = True

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [56]:
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy

initial_learning_rate = 5e-05
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,  
    decay_rate=0.01,    
    staircase=True)
optimizer = Adam(
    learning_rate=lr_schedule,
    epsilon=1e-08,
    clipnorm=1.0
)
loss = CategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [57]:
y_train

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [58]:
r = model.fit(
    x={'input_ids': X_train['input_ids'], 'attention_mask': X_train['attention_mask']},
    y=y_train,
    validation_data=(
        {'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']},
        y_test
    ),
    epochs=2,
    batch_size=32
)

Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:560: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
W0000 00:00:1734522205.644850     108 assert_op.cc:38] Ignoring Assert operator functional_3_1/bert_layer_1_1/tf_bert_model_3/bert/embeddings/assert_less/Assert/Assert


3749/3750 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7079 - loss: 0.7594

W0000 00:00:1734522324.319307     108 assert_op.cc:38] Ignoring Assert operator functional_3_1/bert_layer_1_1/tf_bert_model_3/bert/embeddings/assert_less/Assert/Assert


3750/3750 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7079 - loss: 0.7594

W0000 00:00:1734522331.393299     109 assert_op.cc:38] Ignoring Assert operator functional_3_1/bert_layer_1_1/tf_bert_model_3/bert/embeddings/assert_less/Assert/Assert
W0000 00:00:1734522339.931501     110 assert_op.cc:38] Ignoring Assert operator functional_3_1/bert_layer_1_1/tf_bert_model_3/bert/embeddings/assert_less/Assert/Assert


3750/3750 ━━━━━━━━━━━━━━━━━━━━ 147s 35ms/step - accuracy: 0.7080 - loss: 0.7593 - val_accuracy: 0.8664 - val_loss: 0.3852
Epoch 2/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 121s 32ms/step - accuracy: 0.8647 - loss: 0.3875 - val_accuracy: 0.8702 - val_loss: 0.3705


In [59]:
encoded_dict = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

In [60]:
def predict(text):
    x_val = tokenizer(
        text=text,
        add_special_tokens=True,
        max_length=32,
        truncation=True,
        padding='max_length',
        return_tensors='tf',
        return_token_type_ids=False,
        return_attention_mask=True,
        verbose=True
    )

    validation = model.predict({
        'input_ids': x_val['input_ids'],
        'attention_mask': x_val['attention_mask']
    }) * 100  
    predicted_index = validation[0].argmax()  
    predicted_label = encoded_dict[predicted_index]  
    predicted_score = validation[0][predicted_index] 
    return predicted_label, predicted_score

In [61]:
news_line = "The global economy is recovering as countries start easing restrictions, boosting business confidence and encouraging travel."
predicted_label, predicted_score = predict(news_line)
print(f"Predicted Label: {predicted_label} with score: {predicted_score:.2f}%")

W0000 00:00:1734522476.678792     108 assert_op.cc:38] Ignoring Assert operator functional_3_1/bert_layer_1_1/tf_bert_model_3/bert/embeddings/assert_less/Assert/Assert


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Label: Business with score: 87.95%


In [62]:
news_line = "Scientists have discovered a new method for converting carbon dioxide into useful fuels, which could significantly reduce greenhouse gas emissions and combat climate change."
predicted_label, predicted_score = predict(news_line)
print(f"Predicted Label: {predicted_label} with score: {predicted_score:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted Label: Sci/Tech with score: 93.13%


In [63]:
news_line = "Artificial Intelligence (AI) has become a transformative force in modern science and technology, revolutionizing fields such as healthcare, where it accelerates diagnostics and drug discovery by analyzing vast amounts of data, autonomous vehicles, where AI enables self-driving cars to navigate and make real-time decisions, robotics, quantum computing, and renewable energy, where it optimizes energy grids and predicts consumption, while also raising concerns about ethics, data privacy, job displacement, and the need for responsible development, making AI both a powerful tool and a challenge for the future of technology and society."
predicted_label, predicted_score = predict(news_line)
print(f"Predicted Label: {predicted_label} with score: {predicted_score:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Label: Sci/Tech with score: 96.49%
